# **binlog_cache_size**



- MySQL은 binlog를 기록하는 내용을 캐시하며 그 사이즈를 설정함 (연결된 thread 마다 할당)

- 해당 설정은 트랜잭션에 한하며, statement(non-transactional)에 대한 캐시는  binlog_stmt_cache_size 가 담당함

> STATUS값 확인 : Binlog_stmt_cache_use, Binlog_stmt_cache_disk_use



# **튜닝**

- large transaction을 자주 사용하면 binlog_cache_size를 늘려줘야함.

**status 값을보고 튜닝**

- Binlog_cache_use, Binlog_cache_disk_use
 - 트랜잭션이 binlog_cache_size 보다 크면 해당 트랜잭션은 disk에 기록.

> Binlog_cache_disk_use 수치가 증가하면 디스크에 기록한 횟수가 많은 것이므로 binlog_cache_size를 늘려야 한다. - 캐시히트율 = ((1 - (6031/2854753))*100) = 99.8%


**디스크로 쓰이는 트랜잭션 사이즈는 어떻게 알수 있는가?**

- 수백만건을 update 하는 단일 트랜잭션이 실행된다고 가정
  - 이 내용을 binlog에 쓰기 전에 binlog cache에 먼저 씀.
  - 이때 binlog cache 사이즈보다 크면 tmp 에 파일을 생성.

- lsof(List Open File) 명령어로 tmp에 생성되는 파일을 확인해 볼수 있음.

> lsof : 현재 시스템에서 돌아가는 process에 의해 Open된 파일에 대한 정보를 보여줌

```
$ lsof | grep tmpdir
```



- 비-트랜젝션 (non-transactional) 테이블에 대한 업데이트는 명령문 실행 후에 즉시 바이너리 로그에 저장
- 실행되지 않은 트랜젝션 안에서는, BDB 또는 InnoDB 테이블과 같은 트랜젝션 테이블을 변경하는 모든 업데이트 (UPDATE, DELETE, 또는 INSERT)는 COMMIT 명령문을 서버가 받을 때까지 캐시에 저장
- 그 시점에서, mysqld는 전체 트랜젝션을 COMMIT가 실행되기 전에 바이너리 로그에 기록
- 트랜젝션을 처리하는 쓰레드가 시작되면, 쓰레드는 binlog_cache_size 버퍼를 버퍼 명령문에 할당
- 명령문이 이것보다 크다면, 쓰레드는 트랜젝션을 저장하기 위해 임시 파일을 염
- 임시 파일은 쓰레드가 종료하면 삭제

- 비-트랜젝션 테이블에 대한 수정은 롤백되지 않음
- 롤백된 트랜젝션이 비-트랜젝션 테이블 수정을 포함하고 있다면, 전체 트렌젝션은 이러한 테이블에 대한 수정이 리플리케이션 되었음을 확신하기 위해서 ROLLBACK 명령문을 사용해서 마지막에 기록

- Binlog_cache_use 상태 변수는 명령문을 저장하기 위한 용도로 이 버퍼 (또는 임시 파일)를 사용한 트랜젝션의 숫자를 표시
- Binlog_cache_disk_use 상태 변수는 이 임시 파일을 실제로 사용한 트랜젝션의 숫자를 표시
- 이 두 가지 변수는 임시 파일 사용을 피하기 위한 binlog_cache_size 크기를 튜닝하는데 사용